In [ ]:
# Transformer 모델 구축 - Transformer RAG(Retriever Augmentation Generation) 구성 및 모델 파이프라인 구축
# - Retrieval 단계: 검색엔진이나 벡터DB(예: FAISS, Milvus, Weaviate)를 통해 관련 문서를 빠르게 찾아냄
# - Augmentation 단계: 검색된 문서를 기반으로 LLM 입력 프롬프트를 강화 → 맥락 있는 답변 생성
# - Generation 단계: LLM이 최종 답변을 생성 → 단순 생성보다 정확성과 신뢰성이 높아짐

# 학습 목표 - 실무에서 사용되는 파이프라인 이해 및 적용


In [14]:
# RAG 구성도 및 모델 파이프라인 검토
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import pipeline

# QA 모델( KoELECTRA KorQuAD)
qa_tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-finetuned-korquad")
qa_model = AutoModelForQuestionAnswering.from_pretrained("monologg/koelectra-base-v3-finetuned-korquad")
qa_pipeline = pipeline(
    "question-answering",
    model=qa_model,
    tokenizer=qa_tokenizer
)

# 요약 모델(KoBART)
kobart_tokenizer = BartTokenizer.from_pretrained("gogamza/kobart-summarization")
kobart_model = BartForConditionalGeneration.from_pretrained("gogamza/kobart-summarization")

# LLM 모델(GPT 계열 API 호출 또는 파인튜닝된 모델)
# - 선택 필요


# Hugging Face pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
# 테스트 성능 미달
# llm_tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")
# llm_model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2")
# 리소스 부족
# llm_tokenizer = AutoTokenizer.from_pretrained("beomi/KoAlpaca-Polyglot-12.8B")
# llm_model = AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot-12.8B")

llm_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-1.3b")
llm_model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/polyglot-ko-1.3b",
    torch_dtype=torch.float16,
    device_map="auto",   # GPU 메모리에 맞게 자동 배치
    offload_folder="offload"  # 일부 레이어를 CPU로 오프로딩
)

llm_pipeline = pipeline(
    "text-generation", 
    model=llm_model,
    tokenizer=llm_tokenizer,
    return_full_text=False # 프롬프트 제외
)

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

ElectraForQuestionAnswering LOAD REPORT from: monologg/koelectra-base-v3-finetuned-korquad
Key                             | Status     |  | 
--------------------------------+------------+--+-
electra.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


Loading weights:   0%|          | 0/260 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/292 [00:00<?, ?it/s]

In [24]:
# 파이프라인 실행
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 임베딩 모델 로드
# - 여기서는 paraphrase-multilingual-MiniLM-L12-v2 모델을 사용했는데, 다국어(한국어 포함) 문장 의미를 잘 반영하는 임베딩을 생성한다
# - 문장을 입력하면 의미 공간에서 가까운 벡터로 변환
embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 문서 벡터화 및 DB 구축
documents = [ # 검색 대상이 되는 문서 리스트
    "최근 AI 분야에서는 RAG 시스템이 각광받고 있으며...",
    "AI는 의료, 금융 등 다양한 분야에서 활용되고 있다..."
]
# 각 문서를 벡터로 변환: 예시) "AI는 의료, 금융..." → [0.12, -0.34, ...]
doc_embeddings = embedder.encode(documents)

# Faiss 인덱스 구축
# - faiss.IndexFlatL2() L2 거리(유클리드 거리) 기반 최근접 검색 인덱스, doc_embeddings.shape[1] 벡터 차원 수를 지정
# - faiss 인덱스를 만들때는 벡터의 차원수를 지정해야 한다, doc_embeddings.shape (문서개수, 임베딩 차원수)
index = faiss.IndexFlatL2(doc_embeddings.shape[1]) # doc_embeddings.shape[1] (임베딩 차원수) (예시 384)
# faiss index.add() 는 (N 벡터개수, d 벡터 차원수) 형태의 float32 넘파이 배열을 기대한다 
index.add(np.array(doc_embeddings))

# 검색 단계
query = "AI 최신 동향은 무엇인가요?"
query_embedding = embedder.encode([query]) # (1, 384) 형태의 벡터, 질의 벡터

# index.search()는 질의 벡터와 인덱스에 저장된 문서 벡터들 사이의 L2 거리(유클리드 거리)를 계산
# - D: 거리 값들(작을수록 더 가까움), I: 가장 가까운 벡터의 인덱스 번호
D, I = index.search(np.array(query_embedding), k=1)
# 예시) documents = ["문서0", "문서1"]
search_result = documents[I[0][0]] # I는 가장 가까운 벡터의 인덱스 번호에 맞는 documents[I[0][0]] 문서 인덱스 와 문서
print("search_result:", search_result)

# QA 단계
qa_result = qa_pipeline(
    question=query,
    context=search_result
)
answer_candidate = qa_result["answer"]
print("QA 결과:", answer_candidate)

# 요약 단계(조건부)
if len(search_result.split()) > 50:
    inputs = kobart_tokenizer(
        search_result,
        return_tensors="pt",
        max_length=512,
        truncation=True
    )
    summary_ids = kobart_model.generate(
        inputs=inputs["input_ids"],
        max_length=128,
        num_beams=4,
        early_stopping=True
    )
    summary_text = kobart_tokenizer.decode(
        summary_ids[0],
        skip_special_tokens=True
    )
else:
    summary_text = search_result
print("문서 요약:", summary_text)

# LLM 최종 생성
# llm_input = f"질문: {query}\n문서요약: {summary_text}\nQA 모델 정답 후보: {answer_candidate}\n위 정보를 바탕으로 사용자의 질문에 대해 간결하고 정확한 답변을 작성하세요."
llm_input = f"""
{query}
{summary_text}

위 정보를 참고해 한두 문장으로 답변하세요.
"""


raw_output = llm_pipeline(
    llm_input,
    max_new_tokens=150,
    num_return_sequences=1,
    do_sample=False
)[0]["generated_text"]

# 불필요한 키워드 제거
for kw in ["질문:", "문서 요약:", "요약:"]:
    raw_output = raw_output.replace(kw, "")

final_answer = raw_output.strip()
print("최종 답변:", final_answer)


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


search_result: AI는 의료, 금융 등 다양한 분야에서 활용되고 있다...


Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


QA 결과: 의료, 금융
문서 요약: AI는 의료, 금융 등 다양한 분야에서 활용되고 있다...
최종 답변: AI는 의료, 금융 등 다양한 분야에서 활용되고 있다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​
